In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import sys
from statsmodels.regression.linear_model import OLS
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools import add_constant
import datetime
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
%matplotlib inline

In [ ]:
target_col = "KneeFlex_maxExtension"
sided = "_singlesided"
if target_col in ["KneeFlex_maxExtension"]:
    sided = ""

predictions = pd.read_csv("./data/predictions/cnn_%s%s_predictions_all_epochs.csv" % (target_col,sided),dtype={'videoid':int})

In [ ]:
predictions['const'] = 1

In [ ]:
def get_corrected_model(iteration):
    lm = sm.OLS(predictions[predictions['dataset'] == 'train'][target_col].values,
                           predictions[predictions['dataset'] == 'train'][['%s_pred_%s' % (target_col,iteration),"const"]].values).fit()

    predictions['%s_pred_%s_corrected' % (target_col,iteration)] = lm.predict(predictions[['%s_pred_%s' % (target_col,iteration),"const"]])
    predictions['error2'] = np.square(predictions[target_col] - predictions['%s_pred_%s_corrected' % (target_col,iteration)])
    rmses = np.sqrt(predictions.groupby('dataset')['error2'].mean())
    return lm, rmses

train_rmse = []
val_rmse = []
for i in range(100):
    _, rmses = get_corrected_model(i)
    train_rmse.append(rmses.loc['train'])
    val_rmse.append(rmses.loc['validation'])

In [ ]:
best_epoch = np.argmin(val_rmse)

In [ ]:
print(best_epoch)
lm, _ = get_corrected_model(best_epoch)
filepath = "./data/models/%s_correction.pb" % target_col
lm.save(filepath, remove_data = True)

In [ ]:
predictions['%s_pred_corrected' % (target_col)] = predictions['%s_pred_%s_corrected' % (target_col,best_epoch)]

In [ ]:
side_col = []
if target_col in ["KneeFlex_maxExtension"]:
    side_col = ['side']
predictions[['%s_pred_corrected' % (target_col),'videoid']+side_col+['dataset']].\
    to_csv("./data/predictions/cnn_%s_predictions_best_epoch.csv" % (target_col),index=False)